# Train model in the AID and UCM dataset

This notebook is used to train the final model in the AID and UCM datasets using the pretrained backbone in the HRRS dataset.

## 1. Import modules and define parameters

In [ ]:
# clone repository with the backbone model
!git clone https://github.com/LucasKirsten/InceptionV4_TensorFlow2.git

In [ ]:
# ignore all future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# import necessary modules
import sys
sys.path.append('./InceptionV4_TensorFlow2')
from inception_resnet_v2 import InceptionResNetV2

import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as KL
from keras.engine import data_adapter
from tqdm import tqdm
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score

In [ ]:
# use mixed precision training (i.e., variables in float16 and computation in float32)
# this speeds up the training and allows higher batch sizes
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
# define strategy (single or multi gpu)
if len(tf.config.experimental.list_physical_devices('GPU'))>1:
    strategy = tf.distribute.MirroredStrategy()
    print('Using mirrored strategy')
else:
    strategy = tf.distribute.get_strategy()

In [ ]:
# define the parameters for training
BATCH_SIZE = 64
PATH_TFRECORDS = '../input/aid-dataset/AID_tfrecords/*.tfrec'
LR = 0.001
LR_ATTENUATION = 0.1
TOTAL_STEPS = int(9600) # divide by 5 to UCM
STEPS_PER_EPOCH = int(1920) # divide by 5 to UCM
EPOCHS = int(TOTAL_STEPS/STEPS_PER_EPOCH)
NUM_CLASSES = 30 # 30 for AID and 21 for UCM

## 2. Define DL model

In [ ]:
# define the DL model
def get_model():
    with strategy.scope():
        inputs = KL.Input((256,256,3))
        backbone = InceptionResNetV2()
        # load pretrained weights
        backbone.load_weights('../input/hrrs-pretrained/weights')

        x = backbone(inputs)
        x = KL.GlobalAveragePooling2D() (x)
        x = KL.Dropout(0.2) (x)
        x = KL.Dense(NUM_CLASSES, activation='softmax') (x)

        model = tf.keras.Model(inputs, x)

        # compile model
        model.compile(tf.keras.optimizers.SGD(learning_rate=LR), \ # Adam for better results
                      loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])

        return backbone, model
    
get_model()[1].summary()

## 3. Define data loader

In [ ]:
# function to read data from tfrecords
def _read_data(example):
    LABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = tf.reshape(tf.io.decode_raw(example['image'], tf.uint8), (256,256,3))
    image = tf.cast(image, 'float32')/255.
    label = tf.reshape(tf.io.decode_raw(example['label'], tf.float32), (NUM_CLASSES,))
    return image, label

def get_dataset(paths_tf_records, repeat=True):
    with tf.device('/cpu:0'):
        ignore_order = tf.data.Options()
        ignore_order.experimental_deterministic = False

        dataset = tf.data.TFRecordDataset(paths_tf_records,\
                                          compression_type='GZIP',\
                                          num_parallel_reads=tf.data.experimental.AUTOTUNE)
        dataset = dataset.shuffle(2048)
        dataset = dataset.with_options(ignore_order)
        dataset = dataset.map(_read_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.batch(BATCH_SIZE)
        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        if repeat:
            dataset = dataset.repeat()

        return dataset

## 4. Define training setup

In [ ]:
# learning rate scheduler
def _schedule(epoch, lr):
    return lr*LR_ATTENUATION
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(_schedule, verbose=1)

In [ ]:
# train model
accs,kappas = [],[]
for i in range(5):
    # split into train and test
    train,test = train_test_split(glob(PATH_TFRECORDS), test_size=0.2)
    
    # get reset model
    backbone, model = get_model()
    
    # freeze backbone in first epoch
    backbone.trainable = False
    model.fit(
        get_dataset(train),
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs = 1,
        validation_data = get_dataset(test),
        validation_steps = len(test)//BATCH_SIZE
    )
    
    # unfreeze backbone
    backbone.trainable = True
    model.fit(
        get_dataset(train),
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs = EPOCHS - 1,
        callbacks = [lr_scheduler],
        validation_data = get_dataset(test),
        validation_steps = len(test)//BATCH_SIZE
    )
    
    # evaluate model and store results
    test = get_dataset(test, repeat=False)
    y_true,y_pred = [],[]
    for x,y in iter(test):
        y_pred.append(model(x))
        y_true.append(y)
    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)
    
    y_true,y_pred = np.argmax(y_true,axis=-1),np.argmax(y_pred,axis=-1)
    
    accs.append(accuracy_score(y_true, y_pred))
    kappas.append(cohen_kappa_score(y_true, y_pred))
    
    print(f'Accuracy: {accs[-1]}')
    print(f'Kappa: {kappas[-1]}')
    
    # delete models of this iteration
    del backbone, model
    tf.keras.backend.clear_session()

In [ ]:
print(f'Accuracy: {np.mean(accs)}±{np.std(accs)}')
print(f'Kappa: {np.mean(kappas)}±{np.std(kappas)}')

### Results
| Optimizer | Model | Accuracy | Kappa |
| - | - | - | - |
| SGD  | AID | 90.42±0.10 | 0.90±0.001 |
| SGD  | UCM | 90.55±2.8  | 0.90±0.029 |
| Adam | UCM | 95.77±0.84 | 0.96±0.01 |
| Adam | AID | 93.38±0.27 | 0.93±0.003 |